# I. Data Cleaning

### Script Imports

In [1]:
!pip install nfl-data-py numpy==1.26.4 scipy==1.11.4 scikit-learn==1.4.2 tpot==0.12.2

In [6]:
!python --version

Python 3.12.3


In [7]:
import sys
sys.path.extend(['./extras', './weekly', './models', './utils'])
import pandas as pd
from coach import get_coaches
from down_stats import get_down_stats
from game import get_game_data
from line import get_line_rank

### Get Data from Extras

In [3]:
f_before_2024 = pd.concat([get_coaches(year) for year in range(2022, 2024)], ignore_index=True)
f_2024 = get_coaches(2024)
f_before_2024.rename(columns={'Tm': 'Team'}, inplace=True)
f_2024.rename(columns={'Tm': 'Team'}, inplace=True)
down_before_2024 = pd.concat([get_down_stats(year) for year in range(2022, 2024)], ignore_index=True)
down_2024 = get_down_stats(2024)
game_before_2024 = pd.concat([get_game_data(year) for year in range(2022, 2024)], ignore_index=True)
game_before_2024 = game_before_2024.drop_duplicates(subset=['year', 'week', 'team'], keep='first')
game_2024 = get_game_data(2024)
game_before_2024.rename(columns={'team': 'Team'}, inplace=True)
game_2024.rename(columns={'team': 'Team'}, inplace=True)
line_before_2024 = pd.concat([get_line_rank(2022), get_line_rank(2023)], ignore_index=True)
line_2024 = get_line_rank(2024)

In [4]:
line_before_2024[line_before_2024.isna().any(axis=1)]

,year,Team,oline_rank,dline_rank


In [19]:
line_2024[line_2024.isna().any(axis=1)]

,year,Team,oline_rank,dline_rank


### Merge coaches and down stats

In [20]:
f_before_2024.columns, down_before_2024.columns

(Index(['year', 'week', 'Team', 'Coach', 'G-season', 'W-season', 'L-season',
        'T-season', 'G-w/team', 'W-w/team', 'L-w/team', 'T-w/team', 'G-career',
        'W-career', 'L-career', 'T-career', 'G plyf-playoffs',
        'W plyf-playoffs', 'L plyf-playoffs', 'G plyf-w/team', 'W plyf-w/team',
        'L plyf-w/team', 'G plyf-career', 'W plyf-career', 'L plyf-career',
        'Remark', 'w/same_team'],
       dtype='object'),
 Index(['Team', 'Gms-fds', 'Rush-fds', 'Pass-fds', 'Pen-fds', 'Pct-tde',
        'Pct-fde', 'year'],
       dtype='object'))

In [21]:
f_before_2024 = pd.merge(f_before_2024, down_before_2024, on=['year', 'Team'], how='inner')
f_2024 = pd.merge(f_2024, down_2024, on=['year', 'Team'], how='inner')
print(f_before_2024[f_before_2024.isna().any(axis=1)])
f_2024[f_2024.isna().any(axis=1)]

Empty DataFrame
Columns: [year, week, Team, Coach, G-season, W-season, L-season, T-season, G-w/team, W-w/team, L-w/team, T-w/team, G-career, W-career, L-career, T-career, G plyf-playoffs, W plyf-playoffs, L plyf-playoffs, G plyf-w/team, W plyf-w/team, L plyf-w/team, G plyf-career, W plyf-career, L plyf-career, Remark, w/same_team, Gms-fds, Rush-fds, Pass-fds, Pen-fds, Pct-tde, Pct-fde]
Index: []

[0 rows x 33 columns]


,year,week,Team,Coach,G-season,W-season,L-season,T-season,G-w/team,W-w/team,...,W plyf-career,L plyf-career,Remark,w/same_team,Gms-fds,Rush-fds,Pass-fds,Pen-fds,Pct-tde,Pct-fde


In [22]:
duplicates = f_2024[f_2024.duplicated(subset=['year', 'Coach', 'week', 'Team'], keep=False)]
duplicates

,year,week,Team,Coach,G-season,W-season,L-season,T-season,G-w/team,W-w/team,...,W plyf-career,L plyf-career,Remark,w/same_team,Gms-fds,Rush-fds,Pass-fds,Pen-fds,Pct-tde,Pct-fde


### Merge game

In [23]:
duplicates = game_2024[game_2024.duplicated(subset=['year', 'week', 'Team'], keep=False)]
duplicates

,year,week,Team,gameday,weekday,gametime,away_team,home_team,away_moneyline,home_moneyline,...,under_odds,over_odds,div_game,roof,surface,temp,wind,referee,stadium,bye_week


In [24]:
f_2024.columns, game_2024.columns

(Index(['year', 'week', 'Team', 'Coach', 'G-season', 'W-season', 'L-season',
        'T-season', 'G-w/team', 'W-w/team', 'L-w/team', 'T-w/team', 'G-career',
        'W-career', 'L-career', 'T-career', 'G plyf-playoffs',
        'W plyf-playoffs', 'L plyf-playoffs', 'G plyf-w/team', 'W plyf-w/team',
        'L plyf-w/team', 'G plyf-career', 'W plyf-career', 'L plyf-career',
        'Remark', 'w/same_team', 'Gms-fds', 'Rush-fds', 'Pass-fds', 'Pen-fds',
        'Pct-tde', 'Pct-fde'],
       dtype='object'),
 Index(['year', 'week', 'Team', 'gameday', 'weekday', 'gametime', 'away_team',
        'home_team', 'away_moneyline', 'home_moneyline', 'spread_line',
        'away_spread_odds', 'home_spread_odds', 'total_line', 'under_odds',
        'over_odds', 'div_game', 'roof', 'surface', 'temp', 'wind', 'referee',
        'stadium', 'bye_week'],
       dtype='object'))

In [25]:
f_before_2024 = pd.merge(f_before_2024, game_before_2024, on=['year', 'week', 'Team'], how='inner')
f_2024 = pd.merge(f_2024, game_2024, on=['year', 'week', 'Team'], how='inner')
print(f_before_2024[f_before_2024.isna().any(axis=1)])
f_2024[f_2024.isna().any(axis=1)]

Empty DataFrame
Columns: [year, week, Team, Coach, G-season, W-season, L-season, T-season, G-w/team, W-w/team, L-w/team, T-w/team, G-career, W-career, L-career, T-career, G plyf-playoffs, W plyf-playoffs, L plyf-playoffs, G plyf-w/team, W plyf-w/team, L plyf-w/team, G plyf-career, W plyf-career, L plyf-career, Remark, w/same_team, Gms-fds, Rush-fds, Pass-fds, Pen-fds, Pct-tde, Pct-fde, gameday, weekday, gametime, away_team, home_team, away_moneyline, home_moneyline, spread_line, away_spread_odds, home_spread_odds, total_line, under_odds, over_odds, div_game, roof, surface, temp, wind, referee, stadium, bye_week]
Index: []

[0 rows x 54 columns]


,year,week,Team,Coach,G-season,W-season,L-season,T-season,G-w/team,W-w/team,...,under_odds,over_odds,div_game,roof,surface,temp,wind,referee,stadium,bye_week


In [26]:
duplicates = f_before_2024[f_before_2024.duplicated(subset=['year', 'Coach', 'week', 'Team'], keep=False)]
duplicates

,year,week,Team,Coach,G-season,W-season,L-season,T-season,G-w/team,W-w/team,...,under_odds,over_odds,div_game,roof,surface,temp,wind,referee,stadium,bye_week


### Merge Line

In [27]:
f_2024.columns, line_2024.columns

(Index(['year', 'week', 'Team', 'Coach', 'G-season', 'W-season', 'L-season',
        'T-season', 'G-w/team', 'W-w/team', 'L-w/team', 'T-w/team', 'G-career',
        'W-career', 'L-career', 'T-career', 'G plyf-playoffs',
        'W plyf-playoffs', 'L plyf-playoffs', 'G plyf-w/team', 'W plyf-w/team',
        'L plyf-w/team', 'G plyf-career', 'W plyf-career', 'L plyf-career',
        'Remark', 'w/same_team', 'Gms-fds', 'Rush-fds', 'Pass-fds', 'Pen-fds',
        'Pct-tde', 'Pct-fde', 'gameday', 'weekday', 'gametime', 'away_team',
        'home_team', 'away_moneyline', 'home_moneyline', 'spread_line',
        'away_spread_odds', 'home_spread_odds', 'total_line', 'under_odds',
        'over_odds', 'div_game', 'roof', 'surface', 'temp', 'wind', 'referee',
        'stadium', 'bye_week'],
       dtype='object'),
 Index(['year', 'Team', 'oline_rank', 'dline_rank'], dtype='object'))

In [28]:
f_before_2024 = pd.merge(f_before_2024, line_before_2024, on=['year', 'Team'], how='inner')
f_2024 = pd.merge(f_2024, line_2024, on=['year', 'Team'], how='inner')
print(f_before_2024[f_before_2024.isna().any(axis=1)])
f_2024[f_2024.isna().any(axis=1)]

Empty DataFrame
Columns: [year, week, Team, Coach, G-season, W-season, L-season, T-season, G-w/team, W-w/team, L-w/team, T-w/team, G-career, W-career, L-career, T-career, G plyf-playoffs, W plyf-playoffs, L plyf-playoffs, G plyf-w/team, W plyf-w/team, L plyf-w/team, G plyf-career, W plyf-career, L plyf-career, Remark, w/same_team, Gms-fds, Rush-fds, Pass-fds, Pen-fds, Pct-tde, Pct-fde, gameday, weekday, gametime, away_team, home_team, away_moneyline, home_moneyline, spread_line, away_spread_odds, home_spread_odds, total_line, under_odds, over_odds, div_game, roof, surface, temp, wind, referee, stadium, bye_week, oline_rank, dline_rank]
Index: []

[0 rows x 56 columns]


,year,week,Team,Coach,G-season,W-season,L-season,T-season,G-w/team,W-w/team,...,div_game,roof,surface,temp,wind,referee,stadium,bye_week,oline_rank,dline_rank


In [30]:
duplicates = f_before_2024[f_before_2024.duplicated(subset=['year', 'Coach', 'week', 'Team'], keep=False)]
duplicates

,year,week,Team,Coach,G-season,W-season,L-season,T-season,G-w/team,W-w/team,...,div_game,roof,surface,temp,wind,referee,stadium,bye_week,oline_rank,dline_rank


In [31]:
duplicates = f_2024[f_2024.duplicated(subset=['year', 'Coach', 'week', 'Team'], keep=False)]
duplicates

,year,week,Team,Coach,G-season,W-season,L-season,T-season,G-w/team,W-w/team,...,div_game,roof,surface,temp,wind,referee,stadium,bye_week,oline_rank,dline_rank


### Save Progress

In [32]:
f_before_2024.rename(columns={'Team': 'team'}, inplace=True)
f_2024.rename(columns={'Team': 'team'}, inplace=True)
f_before_2024.to_pickle('pickles/f_before_2024.pickle')
f_2024.to_pickle('pickles/f_2024.pickle')